# gemma 2 2b-it 모델 파인튜닝 해보기
https://devocean.sk.com/blog/techBoardDetail.do?ID=165703&boardType=techBlog

    - data_set : nlpai-lab/KULLM - huggingface
    

### 현재 성능 모니터링
```bash
$ sudo apt install htop  # Ubuntu/Debian 기반
$ htop
```
```bash
$ sudo apt install glances
$ glances
```

In [10]:
! pip install -q -U transformers datasets bitsandbytes peft trl accelerate

In [3]:
# 모듈 정리 
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments, GemmaTokenizerFast
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
# gemma 2 2b-it 모델을 기반하여 학습 시킨다.
BASE_MODEL = "google/gemma-2-2b-it"

### 모델 실행 시 주의 한 번만 실행 요망 ###
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.70s/it]


In [5]:
from transformers import GemmaTokenizerFast

# 토크나이저 결과
tokenizer = GemmaTokenizerFast.from_pretrained("google/gemma-2-2b-it")
tokenizer.encode("Hello this is a test")

[2, 4521, 736, 603, 476, 2121]

## Gemma2 - 2b - it 대화 형식 
```
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
```
    － <bos> ： Beginning of sequence
    － <start_of_turn> ： 시작 구분 기호
    － user : 엔티티의 역할
    － <end_of_turn> : 종료 구분 기호

In [9]:
# 모듈 정리 
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments, GemmaTokenizerFast
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
# 데이터셋 로드 : KULLM이라는 LLM의 학습 데이터셋 불러오기
from datasets import load_dataset

dataset = load_dataset('nlpai-lab/kullm-v2', 
                       split="train")
dataset

/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [4]:
dataset[-1]

{'id': 'vicuna_{idx}',
 'instruction': '항바이러스제는 생명을 위협하는 바이러스에만 사용되나요?',
 'input': '',
 'output': '항바이러스제는 생명을 위협하는 바이러스에만 사용되는 것이 아닙니다. 경증 및 중증 바이러스를 포함한 광범위한 바이러스 감염을 치료하는 데 사용할 수 있습니다. 항바이러스제 사용 여부는 일반적으로 특정 바이러스, 감염의 중증도, 환자의 전반적인 건강 상태에 따라 결정됩니다.예를 들어, 항바이러스제는 인플루엔자(독감) 치료에 자주 사용되는데, 인플루엔자는 노인, 어린이, 특정 기저질환이 있는 사람 등 특정 사람들에게는 심각하고 심지어 생명을 위협하는 질병이 될 수 있습니다. 항바이러스제는 독감의 중증도와 기간을 줄이는 데 도움이 될 수 있으며, 폐렴과 같은 심각한 합병증을 예방하는 데도 도움이 될 수 있습니다.다른 경우에는 항바이러스제를 사용하여 생명을 위협하지는 않지만 심각한 불편함을 유발하고 일상 활동에 지장을 줄 수 있는 바이러스 감염을 치료할 수 있습니다. 예를 들어, 항바이러스제는 피부나 점막에 통증성 궤양을 유발할 수 있는 단순포진 바이러스(HSV) 감염을 치료하는 데 사용될 수 있습니다.전반적으로 바이러스 감염을 치료하기 위해 항바이러스제를 사용할지 여부는 특정 바이러스, 감염의 심각성, 치료의 잠재적 위험과 이점을 포함한 다양한 요인에 따라 결정됩니다.'}

In [5]:
# 질문 / 대답
dataset['instruction'][0], dataset['output'][0]

('3원색이란 무엇인가요?',
 '세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.')

## 학습용 프롬프트 조정

<bos><start_of_turn>user
{본문} <end_of_turn>
<start_of_turn>model
{대답} <end_of_turn>

In [6]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append(r"""<bos><start_of_turn>user
사용자의 질문 입니다. 적절한 답변을 해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['instruction'][i], example['output'][i]))
    return prompt_list

In [10]:
generate_prompt(dataset[:10])

['<bos><start_of_turn>user\n사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n3원색이란 무엇인가요?<end_of_turn>\n<start_of_turn>model\n세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.<end_of_turn><eos>',
 '<bos><start_of_turn>user\n사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n어려운 결정을 내려야 했던 순간을 설명해 주세요.<end_of_turn>\n<start_of_turn>model\nAI 어시스턴트인 저는 저만의 개인적인 경험이나 물리적 존재를 가지고 있지 않습니다. 하지만 알고리즘, 논리, 데이터를 기반으로 의사 결정을 내리도록 프로그래밍되어 있습니다. 상충되는 정보나 여러 가지 가능한 결과가 있을 수 있기 때문에 의사결정이 어려운 상황에 직면할 때가 있습니다. 그런 경우 저는 각 옵션의 위험과 이점을 분석하고 비교하여 원하는 결과를 얻을 가능성이 가장 높은 결론에 도달할 수 있는 능력을 활용합니다.<end_of_turn><eos>',
 '<bos><start_of_turn>user\n사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n원자의 구조를 설명합니다.<end_of_turn>\n<start_of_turn>model\n원자는 모든 물질의 기본 구성 요소이며 양성자, 중성자, 전자의 세 가지 유형의 입자로 이루어져 있습니다. 원자의 구조는 전자 구름으로 둘러싸인 중앙에 핵이 있는 것으로 설명할 수 있습니다.\n\n원자의 핵은 양성자와 중성자로 구성됩니다. 양성자는 양전하를 띠는 입자이고 중성자는 전하를 띠지 않는 중성 입자입니다. 이 두 입자는 원자의 중심에 있으며 원자 질량의 대부분을 차지하는 원자핵에 위치합니다.\n\n원자핵을 둘러싸고 있는 

## gemma 양자화 

In [11]:
# LoRA 설정: 대규모 언어 모델의 특정 레이어에서만 파라미터를 미세 조정하여
# 메모리 사용량을 줄이고 학습 효율성을 높임
lora_config = LoraConfig(
    r=6,  # LoRA에서 사용되는 저차원 공간의 랭크(r) 값. 값이 작을수록 학습할 파라미터가 적어짐
    lora_alpha=8,  # LoRA의 학습 속도 조절을 위한 스케일링 파라미터. 값이 클수록 학습 변동폭이 커짐
    lora_dropout=0.05,  # 드롭아웃 확률 설정 (5%). 과적합 방지를 위해 일부 노드를 무작위로 제외
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],  
    # LoRA가 적용될 트랜스포머 모델의 특정 레이어들 (쿼리, 출력, 키, 값 등의 투영 레이어)
    task_type="CAUSAL_LM",  # Causal Language Modeling 작업 유형 (GPT 모델이 주로 사용)
)

# BitsAndBytes 설정: 모델을 4비트로 양자화하여 메모리 사용량을 줄이고 성능 최적화
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 모델을 4비트로 로드하여 메모리 효율성을 극대화함
    bnb_4bit_quant_type="nf4",  # NF4(Normalized Float 4) 방식의 4비트 양자화 사용
    bnb_4bit_compute_dtype=torch.float16  # 계산에 사용할 데이터 타입을 float16으로 설정 (16비트 부동소수점)
)

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# gemma 2 2b-it 모델을 기반하여 학습 시킨다.
BASE_MODEL = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto"  # 자동으로 GPU에 할당
)

# 입력 텍스트 처리
input_text = "안녕? 내이름이 뭔지 대답 해줘"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")  # 입력을 GPU로 이동

# 모델을 통해 출력 생성
outputs = model.generate(input_ids['input_ids'], max_new_tokens=32)

# 출력 디코딩 및 표시
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.35s/it]


안녕? 내이름이 뭔지 대답 해줘.

**Answer:**  My name is Bard. 



In [17]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, 
                                             device_map="auto",  # 이 줄이 중요
                                             quantization_config=bnb_config,
                                             attn_implementation='eager'
                                             )
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.32s/it]


In [20]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps = 1000,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 152630/152630 [00:36<00:00, 4180.92 examples/s]
/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precisi

In [21]:
trainer.train()

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss
100,2.228700
200,1.658500
300,1.513600
400,1.469200
500,1.446200
600,1.442700
700,1.405800
800,1.428900
900,1.438500
1000,1.432200


TrainOutput(global_step=3000, training_loss=1.4303289031982422, metrics={'train_runtime': 7944.0505, 'train_samples_per_second': 1.511, 'train_steps_per_second': 0.378, 'total_flos': 5.066762968044288e+16, 'train_loss': 1.4303289031982422, 'epoch': 0.07862150298106532})

In [22]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.45s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/transformers/modeling_utils.py:2633: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(
Saving checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.52s/it]


In [3]:
! ls -alh lora_adapter

total 30M
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep 23 13:06 .
drwxrwxr-x 6 ubuntu ubuntu 4.0K Sep 23 13:06 ..
-rw-rw-r-- 1 ubuntu ubuntu 5.0K Sep 23 13:06 README.md
-rw-rw-r-- 1 ubuntu ubuntu  721 Sep 23 13:06 adapter_config.json
-rw-rw-r-- 1 ubuntu ubuntu  30M Sep 23 13:06 adapter_model.safetensors


In [5]:
! ls -alh ./gemma-2b-it-sum-ko

total 4.9G
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep 23 13:06 .
drwxrwxr-x 6 ubuntu ubuntu 4.0K Sep 23 13:06 ..
-rw-rw-r-- 1 ubuntu ubuntu  880 Sep 23 13:06 config.json
-rw-rw-r-- 1 ubuntu ubuntu  187 Sep 23 13:06 generation_config.json
-rw-rw-r-- 1 ubuntu ubuntu 4.7G Sep 23 13:06 model-00001-of-00002.safetensors
-rw-rw-r-- 1 ubuntu ubuntu 230M Sep 23 13:06 model-00002-of-00002.safetensors
-rw-rw-r-- 1 ubuntu ubuntu  24K Sep 23 13:06 model.safetensors.index.json


### Fine-tuned 모델 추론

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline
import torch

BASE_MODEL = "google/gemma-2-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-sum-ko"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 모델을 4비트로 로드하여 메모리 효율성을 극대화함
    bnb_4bit_quant_type="nf4",  # NF4(Normalized Float 4) 방식의 4비트 양자화 사용
    bnb_4bit_compute_dtype=torch.float16  # 계산에 사용할 데이터 타입을 float16으로 설정 (16비트 부동소수점)
)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)


/home/ubuntu/miniconda3/envs/kkw_env1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


In [2]:

text = '오늘의 날씨는 맑았어 좋은 점이 뭘까?'
messages = [
    {
        "role": "user",
        "content": "사용자의 질문 입니다. 적절한 답변을 해주세요:\n\n{}".format(text)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

오늘의 날씨가 맑으면 여러 가지 이점이 있습니다. 햇빛이 많이 들어오는 것은 우리의 기분을 개선하고 에너지를 끌어올릴 수 있습니다. 또한 맑은 날씨는 산책이나 운동을 하기 좋은 날이 되어 활동에 활력을 불어넣을 수 있습니다. 또한 맑은 날씨는 우리의 기분을 개선하고 스트레스를 줄이는 데 도움이 될 수 있습니다.
